# For Kaggle

In [ ]:
# !pip install gdown
# !gdown 1zDCi0nnxjP3so8wPwc5JGIj55lWimFw4

# !gdown 1p4cBOvRvSsUYdRdkjo4CPFiBqHyjw_87

# Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle as pk
from FS.pso import jfs

# Seeds

In [ ]:
seed_value = 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os

os.environ["PYTHONHASHSEED"] = str(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)
np.random.default_rng(seed=seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

# Remove limit on dataframe columns
pd.set_option("display.max_columns", None)

# Utility

In [ ]:
# Helper functions

# Input a series containing X and y to create a windowed dataset
def multi_features_windowed_dataset(series, window_size, horizon, batch_size, shuffle_buffer):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + horizon, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + horizon))
    ds = ds.shuffle(shuffle_buffer_size)
    ds = ds.map(lambda w: (w[:-horizon, :-1], w[-horizon:, -1]))
    ds = ds.batch(batch_size).prefetch(1)
    return ds

# Input a series containing X only to get Kp for the next timestep
def forecast_single(model, series, window_size, batch_size=64):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(batch_size).prefetch(1)
    forecast = model.predict(ds).squeeze()
    return forecast

# Preprocessing

In [ ]:
# df = pd.read_table("./omni2_all_years.dat", sep="\s+",header=None)

In [ ]:
df = pd.read_csv("./omni2_all_years_v2.csv", index_col=0)

In [ ]:
# df = pd.read_csv("./omnii data ready finale.csv", index_col=0)

In [ ]:
# df

In [ ]:
# Drop columns of IDs and other indices
df.drop(
    [
        "ID for IMF SC",
        "ID for SW Plasma SC",
        "DST Index",
        "AE-index",
        "Ap-index",
        "f10.7_index",
        "AL-index",
        "AU-index",
    ],
    inplace=True,
    axis=1,
)

In [ ]:
# Convert year, day, hour to datetime
time = pd.to_datetime(df["Year"] * 1000 + df["Day"], format="%Y%j")
time = time + pd.to_timedelta(df.Hour, unit="h")
df.index = time

# Reduce the number of kp values from 28 to 10
df["Kp"] = df["Kp*10"].apply(lambda x: round(x / 10))

# Drop the old columns
df = df.drop(["Year", "Day", "Hour", "Kp*10"], axis=1)

In [ ]:
# df

In [ ]:
# # average every 3 rows
# df = df.resample("3H").mean()

In [ ]:
# Chosen time range
df = df[(df.index.year > 1975) & (df.index.year < 2023)]

In [ ]:
# df

In [ ]:
replacement = [
    999,
    999.9,
    9999999.0,
    9999.0,
    99.99,
    9.999,
    999.99,
    999999.99,
    99999.99,
    99.9,
]
# Replace missing values with NaN
df.replace(replacement, np.nan, inplace=True)

In [ ]:
# Reference features that were used in some papers
# ref_features = ["Bz,GSM", "Proton density", "Bulk speed", "Field Magnitude Avg", "Sigma-B", "Kp"]
# df = df[ref_features]

In [ ]:
# Plot the distribution of the features
# for col in df.columns:
#     plt.Figure()
#     df[col].plot(figsize=(20, 5))
#     plt.title(col)
#     plt.show()

In [ ]:
# df.isna().sum()

In [ ]:
# Interpolate the missing values
df.interpolate(method="time", limit_direction="both", inplace=True)

In [ ]:
# df.isna().sum()

In [ ]:
# Correlation between features
# corr_matrix = df.corr()

# # Plot the heatmap
# plt.figure(figsize=(16, 10))
# sns.heatmap(corr_matrix, cmap='coolwarm')
# plt.title('Correlation Matrix Heatmap')
# plt.show()

In [ ]:
# corr_matrix["Kp"].abs().sort_values(ascending=False)

In [ ]:
# Drop some redundant features
df.drop(["By,GSM", "Bz,GSM", "Field Magnitude Avg"], axis=1, inplace=True)

In [ ]:
# Highest correlation with Kp
# df.corrwith(df["Kp"]).abs().sort_values(ascending=False)[:7]

In [ ]:
# selected_features = df.corrwith(df["Kp"]).abs().sort_values(ascending=False)[:7].index.values
# df_sub = df[selected_features]

In [ ]:
# Cross correlation between Kp and other features
# lagged_corr = []
# for i in range(1, 365):
#     lagged_corr.append(df.shift(i).corrwith(df["Kp"]).abs())

In [ ]:
# Cross correlation plot
# lagged_corr = pd.DataFrame(lagged_corr)
# title="Cross Correlation with Kp"
# xlabel="Lag hours"
# ylabel="Correlation with Kp"
# lagged_corr.plot(figsize=(16, 10), title=title, xlabel=xlabel, ylabel=ylabel)

In [ ]:
# Cross correlation between Kp and a subset of features
# lagged_corr = pd.DataFrame(lagged_corr)
# title="Cross Correlation with Kp"
# xlabel="Lag hours"
# ylabel="Correlation with Kp"
# lagged_corr[selected_features].plot(figsize=(16, 10), title=title, xlabel=xlabel, ylabel=ylabel)

In [ ]:
# pso_features = [
#     "Magnitude of Average Field vector",
#     "Bx,GSE",
#     "Sigma-B",
#     "Sigma-Bx",
#     "Sigma-By",
#     "Sigma-Bz",
#     "Na/Np",
#     "Sigma-phi-V",
#     "Sigma-theta-V",
#     "PROT Flux  >1 MeV",
#     "PROT Flux  >2 MeV",
#     "PROT Flux  >30 MeV",
#     "PROT Flux  >60 MeV",
#     "PC(N)",
#     "Kp",
# ]

In [ ]:
# Feature indices that were selected by PSO
# pso_features = [3, 6, 10, 11, 12, 13, 19, 24, 25, 31, 32, 35, 36, 38, 40]
# df = df.iloc[:, pso_features]

In [ ]:
# df

In [ ]:
# Train test split
split_year = 2008
train = df[df.index.year < split_year]
test = df[df.index.year >= split_year]

In [ ]:
# Run PSO
# # parameter
# k    = 5     # k-value in KNN
# N    = 10    # number of particles
# T    = 100   # maximum number of iterations
# fold = fold = {'xt': train.to_numpy(), 'yt':train["Kp"].to_numpy(), 'xv':test.to_numpy(), 'yv':test["Kp"].to_numpy()}
# opts = {'k':k, 'fold':fold, 'N':N, 'T':T}

# # perform feature selection
# fmdl = jfs(df.to_numpy(), df["Kp"].to_numpy(), opts)

In [ ]:
# pso_features = fmdl['sf']

In [ ]:
num_features = df.shape[1]
num_features

In [ ]:
train_y = train["Kp"]
test_y = test["Kp"]

In [ ]:
# Scale the data except for Kp
scaler = StandardScaler()
train[train.columns[:-1]] = scaler.fit_transform(train[train.columns[:-1]])
test[test.columns[:-1]] = scaler.transform(test[test.columns[:-1]])

In [ ]:
# Duplicate the Kp column for the windowed dataset
train_xy = pd.concat([train, train_y], axis=1)
test_xy = pd.concat([test, test_y], axis=1)

In [ ]:
# Run PCA on all features to determine the number of components
# n_components = num_features
# pca = PCA(n_components=n_components)
# pca.fit(train)

In [ ]:
# plt.plot(np.cumsum(pca.explained_variance_ratio_))

In [ ]:
# pca.explained_variance_ratio_[0:30].sum()

In [ ]:
# Fit PCA with chosen number of components
# pca = PCA(n_components=30)
# pca.fit(train)

# train = pca.transform(train)
# test = pca.transform(test)

# train_xy = np.append(train, train_y.values.reshape(-1, 1), axis=1)
# test_xy = np.append(test, test_y.values.reshape(-1, 1), axis=1)

# num_features = pca.n_components_

In [ ]:
# Hyperparameters for the dataset and model
window_size = 24 # window size in hours
horizon = 3 # forecast horizon in hours
batch_size = 64
shuffle_buffer_size = 1000

In [ ]:
# Create windowed datasets
ds = multi_features_windowed_dataset(
    train_xy, window_size, horizon, batch_size=batch_size, shuffle_buffer=shuffle_buffer_size
)
test_ds = multi_features_windowed_dataset(
    test_xy, window_size, horizon, batch_size=batch_size, shuffle_buffer=shuffle_buffer_size
)

# Model

In [ ]:
# Model
model = tf.keras.Sequential(
    [
        tf.keras.layers.GRU(
            100,
            input_shape=(window_size, num_features),
            return_sequences=True,
        ),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.GRU(
            100,
            input_shape=(window_size, num_features),
        ),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(
            32,
            activation="relu",
        ),
        tf.keras.layers.Dense(horizon),
    ]
)
optimizer = tf.keras.optimizers.Adam()
model.compile(loss="mse", optimizer=optimizer)
history = model.fit(ds, epochs=20, validation_data=test_ds, verbose=1)

In [ ]:
# plot loss
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="test")
plt.legend()
plt.show()

# Evaluation

In [ ]:
# Test the model on test data
pred = forecast_single(model, test, window_size)
pred_discrete = pred.round()
pred_discrete = np.clip(pred_discrete, 0, 9)
# Remove the last prediction as it does not have a corresponding true value
pred_discrete = pred_discrete[:-1]

In [ ]:
# Cut the window size from the true values as it does not have a corresponding prediction
real = test_y[window_size:]

# Binary values to classify storms
real_binary = (real < 5).astype(int)
pred_binary = (pred_discrete < 5).astype(int)

In [ ]:
# Plot results for 1 horizon value
x = test_y.index[window_size:]
true_interval = real
pred_interval = pred_discrete[:,0]

plt.plot(x, true_interval, label="True", color="blue")
plt.plot(x, pred_interval, label="Prediction", color="red")
plt.legend()

In [ ]:
# Metrics
r2_scores = []
f1_scores = []
rmses = []
rmses.append(root_mean_squared_error(real, pred_discrete[:, 0]))
r2_scores.append(r2_score(real, pred_discrete[:, 0]))
f1_scores.append(
    classification_report(real_binary, pred_binary[:, 0], output_dict=True)["0"][
        "f1-score"
    ]
)
# Metrics for multiple horizon values
for h in range(1, horizon):
    rmses.append(root_mean_squared_error(real[h:], pred_discrete[:-h, h]))
    r2_scores.append(r2_score(real[h:], pred_discrete[:-h, h]))
    f1_scores.append(
        classification_report(real_binary[h:], pred_binary[:-h, h], output_dict=True)[
            "0"
        ]["f1-score"]
    )

In [ ]:
print(rmses)
print(r2_scores)
print(f1_scores)
# Average metrics for all horizon values
print(f"RMSE: {np.mean(rmses)}")
print(f"R2 Score: {np.mean(r2_scores)}")
print(f"F1 Score: {np.mean(f1_scores)}")

In [ ]:
# plt.plot(r2_scores)

In [ ]:
# plt.plot(f1_scores)

In [ ]:
# Confusion matrix for 1 horizon value
cm = confusion_matrix(real, pred_discrete[:, 0])
sns.heatmap(cm, annot=True, fmt="d")

# Classification report

print(classification_report(real, pred_discrete[:, 0]))

In [ ]:
# Confusion matrix for binary classification
cm = confusion_matrix(real_binary, pred_binary[:, 0])
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    xticklabels=["Storm", "Safe"],
    yticklabels=["Storm", "Safe"],
).set(xlabel="Predicted", ylabel="Real")

print(classification_report(real_binary, pred_binary[:, 0]))

# For future model usage

In [ ]:
# model.save("./model.keras")
# with open('./pca.pkl', 'wb') as pickle_file:
#         pk.dump(pca, pickle_file)
# with open('./standard_scaler.pkl', 'wb') as pickle_file:
#         pk.dump(scaler, pickle_file)

# # returns Kp(np.array), storm(bool)
# def forecast(scaler, pca, model, data):
#     # (remove these : [
#     #     "ID for IMF SC",
#     #     "ID for SW Plasma SC",
#     #     "DST Index",
#     #     "AE-index",
#     #     "Ap-index",
#     #     "f10.7_index",
#     #     "AL-index",
#     #     "AU-index",
#     #     "By,GSM",
#     #     "Bz,GSM", 
#     #     "Field Magnitude Avg"
#     # ])
#     # Average the data to be 3 hourly
#     # (locate kp in the data here)
#     kp = round(kp/10)
#     scaled = scaler.transform(data) # do NOT scale Kp
#     pca_scaled = pca.transform(scaled)
#     pred = model.predict(pca_scaled)
#     pred = pred.round()
#     pred = np.clip(pred_discrete, 0, 9)
#     storm = (pred >= 5).any()
#     return (pred, storm)